In [50]:
import os
import json
import uuid
from typing import Dict, Any, Optional, List,TypedDict, Annotated

from datetime import datetime


# Pydantic models for structured data
from pydantic import BaseModel, Field, field_validator
import google.generativeai as genai
import requests
from sympy import symbols, sympify, diff, integrate

# Modern LangChain imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.messages import HumanMessage, AIMessage
from langchain import hub

In [19]:
# ============================================================================
# PYDANTIC PARSERS (Task 1: IMPLEMENT PARSERS)
# ============================================================================

class WeatherRequest(BaseModel):
    """Parser for weather requests"""
    city: str = Field(..., min_length=1, description="City name for weather query")
    units: Optional[str] = Field("metric", description="Temperature units (metric/imperial)")
    
    @field_validator('city')
    @classmethod
    def validate_city(cls, v):
        if not v.strip():
            raise ValueError("City name cannot be empty")
        return v.strip().title()

class MathExpression(BaseModel):
    """Parser for mathematical expressions"""
    expression: str = Field(..., description="Mathematical expression to compute")
    operation: str = Field(..., description="Type of operation: derivative or integral")
    variable: str = Field("x", description="Variable with respect to which to compute")
    
    @field_validator('operation')
    @classmethod
    def validate_operation(cls, v):
        if v.lower() not in ['derivative', 'integral']:
            raise ValueError("Operation must be 'derivative' or 'integral'")
        return v.lower()

class ConversationContext(BaseModel):
    """Parser for conversation context"""
    user_input: str
    session_id: str
    timestamp: datetime = Field(default_factory=datetime.now)
    intent: Optional[str] = None
    
class AgentResponse(BaseModel):
    """Structured response from agent"""
    content: str
    tool_used: Optional[str] = None
    confidence: Optional[float] = Field(None, ge=0.0, le=1.0)
    metadata: Dict[str, Any] = Field(default_factory=dict)

In [20]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

# ============================================================================
# API SETUP
# ============================================================================

import os
import google.generativeai as genai

# Load Google API key
with open("Google_API_Key.txt") as f:
    google_api_key = f.read().strip()

if google_api_key:
    print("Google API Key loaded:", google_api_key[:5] + "...")
    os.environ["GOOGLE_API_KEY"] = google_api_key
    genai.configure(api_key=google_api_key)
else:
    print("Google API Key not loaded!")

Google API Key loaded: AIzaS...


In [21]:
import os
from langsmith import Client

# Read the API key from the text file
try:
    with open("Langsmith_Key.txt", "r") as f:
        api_key = f.read().strip()
    print("Successfully read API key from file")
except FileNotFoundError:
    print("Error: Langsmith_Key.txt file not found!")
    print("Please create a file named 'Langsmith_Key.txt' with your API key")
    api_key = None
except Exception as e:
    print(f"Error reading key file: {e}")
    api_key = None

# Set environment variables (these will persist for the Jupyter session)
if api_key:
    os.environ["LANGCHAIN_API_KEY"] = api_key
    os.environ["LANGCHAIN_PROJECT"] = "Trial_"
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    
    print("Environment variables set:")
    print("  LANGCHAIN_API_KEY:", repr(os.getenv("LANGCHAIN_API_KEY")[:10] + "..." if len(os.getenv("LANGCHAIN_API_KEY", "")) > 10 else os.getenv("LANGCHAIN_API_KEY")))
    print("  LANGCHAIN_PROJECT:", repr(os.getenv("LANGCHAIN_PROJECT")))
    print("  LANGCHAIN_TRACING_V2:", repr(os.getenv("LANGCHAIN_TRACING_V2")))
    
    # Test connection to LangSmith
    try:
        print("\nAttempting to connect to LangSmith...")
        client = Client()
        projects = list(client.list_projects())
        print("Successfully connected to LangSmith!")
        print("Available projects:", [p.name for p in projects])
        
        # Verify the TestProject exists or create it
        project_names = [p.name for p in projects]
        if "TestProject" not in project_names:
            print("Creating 'TestProject'...")
            client.create_project(project_name="Trial_agent_c")
            print("Created 'Trial_agent_c'")
        else:
            print("'Trial_agent_c' already exists")
            
    except Exception as e:
        print(f"Failed to connect to LangSmith: {e}")
        print("Please check your API key and internet connection")
else:
    print("Cannot proceed without valid API key")

Successfully read API key from file
Environment variables set:
  LANGCHAIN_API_KEY: 'lsv2_pt_5d...'
  LANGCHAIN_PROJECT: 'Trial_'
  LANGCHAIN_TRACING_V2: 'true'

Attempting to connect to LangSmith...
Successfully connected to LangSmith!
Available projects: ['Trial_', 'TestProject', 'default']
'Trial_agent_c' already exists


In [22]:
# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-pro", temperature=0)

In [ ]:
import re
from langsmith import traceable
# ============================================================================
# HELPER FUNCTION FOR MATH PARSING
# ============================================================================

def insert_implicit_multiplication(expr: str) -> str:
    expr = re.sub(r'(?<=\d)(?=[a-zA-Z])', '*', expr)
    expr = re.sub(r'(?<=\))(?=\d|\w)', '*', expr)
    expr = re.sub(r'(?<=[a-zA-Z])(?=\()', '*', expr)
    return expr

# ============================================================================
# TOOLS WITH PYDANTIC VALIDATION
# ============================================================================

@traceable
@tool
def compute_derivative(expression: str) -> str:
    """Compute the derivative of a mathematical expression with respect to x."""
    try:
        math_expr = MathExpression(expression=expression, operation="derivative")
        
        cleaned_expr = insert_implicit_multiplication(math_expr.expression.lower())
        variable = math_expr.variable.lower()
        
        x = symbols(variable)
        expr = sympify(cleaned_expr)
        derivative = diff(expr, x)
        
        return f"The derivative of {math_expr.expression} is: {derivative}"
    except Exception as e:
        return f"Error computing derivative: {e}"

@traceable
@tool
def compute_integral(expression: str) -> str:
    """Compute the indefinite integral of a mathematical expression with respect to x."""
    try:
        math_expr = MathExpression(expression=expression, operation="integral")
        
        cleaned_expr = insert_implicit_multiplication(math_expr.expression.lower())
        variable = math_expr.variable.lower()
        
        x = symbols(variable)
        expr = sympify(cleaned_expr)
        integral = integrate(expr, x)
        
        return f"The integral of {math_expr.expression} is: {integral} + C"
    except Exception as e:
        return f"Error computing integral: {e}"

@traceable
@tool
def weather_tool(city: str) -> str:
    """Get current weather information for a specified city using WeatherAPI.com."""
    try:
        with open("Weather_API_Key.txt") as f:
            WEATHER_API_KEY = f.read().strip()

        base_url = "http://api.weatherapi.com/v1/current.json"
        params = {
            "key": WEATHER_API_KEY,
            "q": city
        }

        response = requests.get(base_url, params=params)

        if response.status_code != 200:
            return f"Error {response.status_code}: Couldn't fetch weather for {city}.\n{response.text}"

        current = response.json()["current"]
        weather_desc = current["condition"]["text"]
        temp_c = current["temp_c"]
        humidity = current["humidity"]
        wind_kph = current["wind_kph"]

        return (f"Weather in {city}: {weather_desc}, "
                f"temperature {temp_c}°C, humidity {humidity}%, "
                f"wind speed {wind_kph} kph.")
    except Exception as e:
        return f"Error fetching weather: {e}"

@traceable
@tool
def general_chat(query: str) -> str:
    """Handle general conversation and questions."""
    try:
        context = ConversationContext(
            user_input=query,
            session_id="default",  # Replace with dynamic if needed
            intent="general_chat"
        )
        # Assumes `llm` is globally defined (e.g., Gemini instance)
        response = llm.invoke(context.user_input)
        return response.content
    except Exception as e:
        return f"Error in general chat: {e}"

In [34]:
# ============================================================================
# CONDITIONAL ROUTING (Task 2: CONDITIONAL NODES AND EDGES)
# ============================================================================

class RouteDecision(BaseModel):
    """Decision model for routing user queries"""
    intent: str = Field(..., description="Detected intent: weather, math, or general")
    confidence: float = Field(..., ge=0.0, le=1.0, description="Confidence in routing decision")
    tool_name: str = Field(..., description="Tool to use for this intent")
    
    @field_validator('intent')
    @classmethod
    def validate_intent(cls, v):
        valid_intents = ['weather', 'math_derivative', 'math_integral', 'general']
        if v not in valid_intents:
            raise ValueError(f"Intent must be one of {valid_intents}")
        return v

def route_query(query: str) -> RouteDecision:
    """Conditional router that decides which tool to use based on query analysis."""
    query_lower = query.lower()
    
    # Weather routing
    weather_keywords = ['weather', 'temperature', 'rain', 'sunny', 'cloudy', 'forecast']
    if any(keyword in query_lower for keyword in weather_keywords):
        return RouteDecision(
            intent="weather",
            confidence=0.9,
            tool_name="weather_tool"
        )
    
    # Math routing
    derivative_keywords = ['derivative', 'differentiate', 'diff', "d/dx"]
    integral_keywords = ['integral', 'integrate', 'antiderivative']
    
    if any(keyword in query_lower for keyword in derivative_keywords):
        return RouteDecision(
            intent="math_derivative",
            confidence=0.85,
            tool_name="compute_derivative"
        )
    
    if any(keyword in query_lower for keyword in integral_keywords):
        return RouteDecision(
            intent="math_integral",
            confidence=0.85,
            tool_name="compute_integral"
        )
    
    # Default to general chat
    return RouteDecision(
        intent="general",
        confidence=0.7,
        tool_name="general_chat"
    )


In [35]:
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

# Wrap each function in a Tool object
compute_derivative_tool = Tool(
    name="compute_derivative",
    func=compute_derivative,
    description="Computes derivatives",
)

compute_integral_tool = Tool(
    name="compute_integral",
    func=compute_integral,
    description="Computes integrals",
)

weather_tool = Tool(
    name="weather_tool",
    func=weather_tool,  # assuming you have a get_weather() function
    description="Provides current weather information",
)

general_chat_tool = Tool(
    name="general_chat",
    func=general_chat,  # assuming you have this function
    description="Handles general conversation",
)

# List of tools passed to the agent
tools = [
    compute_derivative_tool,
    compute_integral_tool,
    weather_tool,
    general_chat_tool,
]

conversational_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [36]:
# ============================================================================
# MAIN CONVERSATION LOOP
# ============================================================================

def run_conversation():
    """Main conversation loop with structured parsing and routing."""
    session_id = str(uuid.uuid4())
    print("Multi-Tool Agent Ready! (Type 'exit' to quit)")
    print("I can help with: weather, calculus (derivatives/integrals), and general chat")
    
    while True:
        try:
            user_input = input("\nYou: ")
            if user_input.lower() in ["exit", "quit", "q"]:
                print("Bot: Goodbye! Have a great day.")
                break
            
            # Parse user input with Pydantic
            context = ConversationContext(
                user_input=user_input,
                session_id=session_id
            )
            
            # Get routing decision
            route_decision = route_query(user_input)
            print(f"Routing: {route_decision.intent} (confidence: {route_decision.confidence:.2f})")
            
            # Execute with agent
            response = conversational_agent.invoke(
                {"input": user_input},
                config={"configurable": {"session_id": session_id}}
            )
            
            # Parse response with Pydantic
            agent_response = AgentResponse(
                content=response.get("output", ""),
                tool_used=route_decision.tool_name,
                confidence=route_decision.confidence,
                metadata={"session_id": session_id, "timestamp": datetime.now().isoformat()}
            )
            
            print(f"Bot: {agent_response.content}")
            
        except Exception as e:
            print(f"Error: {e}")
            continue

if __name__ == "__main__":
    run_conversation()

Multi-Tool Agent Ready! (Type 'exit' to quit)
I can help with: weather, calculus (derivatives/integrals), and general chat
Routing: weather (confidence: 0.90)


> Entering new AgentExecutor chain...
The user is asking for three things: the weather in Seattle, the derivative of 2x, and the integral of 2x. I need to use three different tools to answer this. I will start with the weather.
Action: weather_tool
Action Input: Seattle
Observation: Weather in Seattle: Partly cloudy, temperature 14.4°C, humidity 90%, wind speed 9.0 kph.
Thought:The user is asking for three things: the weather in Seattle, the derivative of 2x, and the integral of 2x. I need to use three different tools to answer this. I will start with the weather.
Action: weather_tool
Action Input: Seattle
Observation: Weather in Seattle: Partly cloudy, temperature 14.4°C, humidity 90%, wind speed 9.0 kph.
Thought:I have already obtained the weather in Seattle. Now I need to compute the derivative of 2x.
Action: compute_derivat

Seattle weather, derive and integrate 2X (sample prompt)

7/24/2025 - Implementations

- langsmith and traces fully implemented (no more connection error)
- hardcoded a fix to the parsing error using regex (personal use but still useful and a learning moment)
- weather API functions correctly
- added conditional routing

TODAYS errors:

1) fixed had to do with out of date implementation

2) Error computing derivative: Sympify of expression 'could not parse '2x'' failed, because of exception being raised:
SyntaxError: invalid syntax (<string>, line 1)

double check the parsing on the math and how to make it easier for the model to ingest use cases that don't use the symbolic math...

try to make it understand that when two values are together in terms of a function that means multiplication; the tool wants you specifically set the 2*X instead of 2X... (small fix do it later!)

fixed with regex and forcing the input to be lower case Sympy is case sensitive; depending on use case that change might need to get reverted!

3) errors with langgraph, there seems to be some places where invoke was not in fact invoked!








Implementing LangGraphStudio